# Initial Data Analysis - Capstone Project

In [5]:
# Imports
import os
import csv
import numpy as np
import pandas as pd
from pandas import Int64Index
import pickle
import sys

In [6]:
def save_as_pkl(object, path):
	pickle.dump(object, open(path, "wb"))

def load_pkl(path):
	obj = pickle.load(open(path, "rb"))
	return obj

### Preprocessing and loading the data

In [108]:
# Processing the .csvs into dataframes and saving them as pickles for easier
# loading on future runs
files = [f for f in os.listdir("data/") if f.split('1')[0] == 'file']
for file in files:
    print ('\n'+file)
    fName = file.split('.')[0]
    df = pd.read_csv('data/' + file, delimiter=',', na_values=['NA'])
#     df["Id"] = df.reset_index().index
#     df.set_index("Id")
    print(df.shape)
    save_as_pkl(df, 'pickles/'+fName+'.pkl')


file10.csv


/Users/janollers/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (35,41,44,47,50,59) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(3979, 392)

file11.csv


/Users/janollers/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (41,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(3958, 392)

file12.csv


...

(4286, 392)

file13.csv


/Users/janollers/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (32,35,38,44,50,56,354,356,370,372) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(4638, 392)

file14.csv


/Users/janollers/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (32,38,41,56) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(4836, 392)

file15.csv
(5557, 392)

file16.csv
(5456, 392)

file17.csv
(5207, 392)


In [79]:
## Loading preprocessed dataframes
pklFiles = [f for f in os.listdir("pickles/") if f.split('1')[0] == 'file']
for file in pklFiles:
	df = load_pkl("pickles/" + file)
	print(df.shape)

(3979, 392)
(3958, 392)
(4286, 392)
(4638, 392)
(4836, 392)
(5557, 392)
(5456, 392)
(5207, 392)


# Change this 'fname' value to load data for different years

In [87]:
# YEAR:
fname = "file16.pkl"
df = load_pkl("pickles/"+fname)

### Starting to analyze the data

In [107]:
# First 30 columns:
firstPart = df.iloc[:,0:30]
# print(firstPart.columns.values)

# Columns 30 to 65: (top 6 course marks)
top6CourseMarks = df.iloc[:,30:66]
# print(top6CourseMarks.columns.values)

cols = []
for column in top6CourseMarks[top6CourseMarks.columns[2::3]]:
    cols.append(df[column])
hstack = pd.concat([x for x in cols], axis=1)
# hstack.fillna(0, inplace=True)
hstack['Sum'] = hstack.mean(axis=1)
print(hstack)

# Columns 66 to 71:
middle = df.iloc[:,66:72]
print(middle["WAVERG1"], middle["WAVERG2"])
# print(middle.columns.values)

# Columns 72 to 391:
choices = df.iloc[:, 72:]
# print(choices.columns.values)

      SECORM1  SECORM2  SECORM3  SECORM4  SECORM5  SECORM6  SECORM7  SECORM8  \
0        93.0     95.0     91.0     87.0     88.0     88.0     87.0     85.0   
1        72.0     88.0     95.0     90.0     94.0     80.0     95.0      NaN   
2        71.0     66.0     70.0     71.0     67.0     81.0     82.0      NaN   
3        83.0     94.0     94.0     90.0     87.0     92.0      NaN      NaN   
4        91.0     93.0     96.0     95.0     96.0     93.0      NaN      NaN   
...       ...      ...      ...      ...      ...      ...      ...      ...   
5451      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
5452     87.0     90.0     77.0     80.0     81.0     87.0      NaN      NaN   
5453     95.0     91.0     88.0     78.0     75.0     96.0     91.0     85.0   
5454     60.0     51.0     77.0     86.0     77.0     86.0     80.0     85.0   
5455     68.0     75.0     82.0      NaN     84.0     76.0     70.0      NaN   

      SECORM9  SECORM10  SECORM11  SECO

In [89]:
# # COlumns 72 to 391:
# choices = df.iloc[:, 72:]
# print(choices.columns.values)
# for col in choices.columns.values:
#     print[]
    
# cols = []
# for column in top6CourseMarks[top6CourseMarks.columns[2::3]]:
#     cols.append(df[column])
# hstack = pd.concat([x for x in cols], axis=1)
# # hstack.fillna(0, inplace=True)
# hstack['Sum'] = hstack.mean(axis=1)
# print(hstack)


# TODOS:
- Determine the cutoff by finding lowest GPA student that got into Mac program
- Group choices for each student
- Divide dataset into those who got into Mac, and those who did not.

# Determining grade cutoff

### Seperating relevant data
- Splitting data into students who accept and offer from mac and those who don't.
- Isolating 8 initial columns of interest for analysis

In [102]:
## Starting columns of interest:
# RESPROV - Province of Residence
# RESCNTY - County of Residence
# ZIP3 - Residence Postal Code (First 3 Digits)
# CONFUNI - Confirmed University (OurUni='196')
# CONFPR - Confirmed Program (OurProg='SI', OurProg_coop='SIA')
# CONFCHOIC - OUAC Confirmed Choice Preference
# WAVERG1 - Weighted Average (best 6 OAC / Senior Level current year finals)
# WAVERG2 - Weighted Average (best 6 OAC / Senior Level all year finals)
## First 6 columns are in first half, last 2 are in "Middle" dataframe.

COIsFirstHalf = ["RESPROV", "RESCNTY", "ZIP3", "CONFUNI", "CONFPR", "CONFCHOIC"]
COIsSecondHalf = ["WAVERG1", "WAVERG2"]

# Adding columns of interest found in first half of the data
columnsOfInterest = pd.concat([firstPart[x] for x in COIsFirstHalf], axis=1)

# Adding the rest of the columns of interest
columnsOfInterest = pd.concat([columnsOfInterest] + [middle[x] for x in COIsSecondHalf],axis=1)
copy = columnsOfInterest.copy()

mcmasterVector = copy["CONFUNI"] == 196
notmcmasterVector = copy["CONFUNI"] != 196

### Some min/max/average output for both student types:

In [106]:
# Students that received and accepted a McMaster offer:
acceptedMcMaster = copy[mcmasterVector]
# print(acceptedMcMaster)
# print(acceptedMcMaster[(acceptedMcMaster["WAVERG2"] > 0) & (acceptedMcMaster["CONFPR"] == "SIA")].min())
print("Students who accepted an offer from McMaster:")
print("Mean values for each column:")
print(acceptedMcMaster[(acceptedMcMaster["WAVERG2"] > 0) & (acceptedMcMaster["CONFPR"] == "SIA")].mean())
print("\nMax values for each column:")
print(acceptedMcMaster[(acceptedMcMaster["WAVERG2"] > 0) & (acceptedMcMaster["CONFPR"] == "SIA")].max())
print("\nLowest 5 weighted average:")
print(acceptedMcMaster[(acceptedMcMaster["WAVERG2"] > 0) & (acceptedMcMaster["CONFPR"] == "SI")].nsmallest(5,"WAVERG2"))
print("\nShape of acceptedMcMaster df (number of students that accepted):")
print(acceptedMcMaster.shape)

# Students that didn't accept a mcmaster offer:
print('\n\n\n\n')
notMcMaster = copy[notmcmasterVector]
# print(notMcMaster)
print("Min average of students who didn't accept mac offer")
print(notMcMaster[notMcMaster["WAVERG2"] > 0].min())
print("\nMax average of students who didn't accept mac offer")
print(notMcMaster[notMcMaster["WAVERG2"] > 0].max())
print("\nAverage average of students who didn't accept mac offer")
print(notMcMaster[notMcMaster["WAVERG2"] > 0].mean())


Students who accepted an offer from McMaster:
Mean values for each column:
RESPROV      124.977901
RESCNTY      149.314917
CONFUNI      196.000000
CONFCHOIC      2.217311
WAVERG1      721.955801
WAVERG2      910.149171
dtype: float64

Max values for each column:
RESPROV      125
RESCNTY      185
CONFUNI      196
CONFPR       SIA
CONFCHOIC     10
WAVERG1      988
WAVERG2      990
dtype: object

Lowest 5 weighted average:
      RESPROV  RESCNTY ZIP3  CONFUNI CONFPR  CONFCHOIC  WAVERG1  WAVERG2
5373      125      144  M7A    196.0     SI        2.0      858      858
4588      125      143  N7Q    196.0     SI        3.0      860      860
135       125      165  M3B    196.0     SI        1.0      825      862
299       125      146  M7I    196.0     SI        1.0      867      867
4066      125      146  M7K    196.0     SI        1.0      867      867

Shape of acceptedMcMaster df (number of students that accepted):
(943, 8)





Min average of students who didn't accept mac offer
RESPRO